In [1]:
%load_ext autoreload
%autoreload 2
from mtsccleav import *

In [2]:
from aeon.utils.conversion import convert_collection
from aeon.utils.validation import get_type
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from aeon.transformations.collection.convolution_based import Rocket, MiniRocket, MultiRocket, HydraTransformer
from sklearn.metrics import accuracy_score

In [3]:
from aeon.classification.convolution_based import (
    Arsenal,
    HydraClassifier,
    MiniRocketClassifier,
    MultiRocketClassifier,
    MultiRocketHydraClassifier,
    RocketClassifier,
)

In [4]:
ts_panel = load_as_pd_multiindex("../data/single_multi_eq.csv")

In [5]:
ts_panel

five_p_cleav_1  five_p_cleav_compl_1  five_p_cleav_2  \
instances time_points                                                         
0         0                        -1                     1               0   
          1                         1                    -1               0   
          2                        -1                     1               0   
          3                         1                    -1               0   
          4                         0                     0               1   
...                               ...                   ...             ...   
826       9                         1                     0               0   
          10                        0                     0              -1   
          11                       -1                     0               0   
          12                        0                     0               1   
          13                       -1                     0               0   

                       five_p_cleav_compl_2  five_p_cleav_prob  \
instances time_points                                            
0         0                               0           0.999468   
          1                               0           0.998997   
          2                               0           0.998971   
          3                               0           0.999318   
          4                              -1           0.999742   
...                                     ...                ...   
826       9                               0           0.000000   
          10                              0           0.000000   
          11                              0           0.000000   
          12                             -1           0.330473   
          13                              1           0.300819   

                       five_p_cleav_ss  five_p_non_cleav_1  \
instances time_points                                        
0         0                          1                   1   
          1                          1                   0   
          2                          1                   0   
          3                          1                  -1   
          4                          1                  -1   
...                                ...                 ...   
826       9                          0                   0   
          10                         0                   0   
          11                         0                   0   
          12                         1                   0   
          13                         1                   1   

                       five_p_non_cleav_compl_1  five_p_non_cleav_2  \
instances time_points                                                 
0         0                                  -1                   0   
          1                                   0                   1   
          2                                  -1                   1   
          3                                   1                   0   
          4                                   1                   0   
...                                         ...                 ...   
826       9                                   0                  -1   
          10                                  0                  -1   
          11                                  0                   1   
          12                                  0                  -1   
          13                                 -1                   0   

                       five_p_non_cleav_compl_2  ...  three_p_cleav_2  \
instances time_points                            ...                    
0         0                                   0  ...                1   
          1                                  -1  ...                0   
          2                                   0  ...                1   
          3                             

In [6]:
X = construct_X(ts_panel, include_five_p_cleav=True, include_prob=True, include_ss=True)
X 

five_p_cleav_1  five_p_cleav_compl_1  five_p_cleav_2  \
instances time_points                                                         
0         0                        -1                     1               0   
          1                         1                    -1               0   
          2                        -1                     1               0   
          3                         1                    -1               0   
          4                         0                     0               1   
...                               ...                   ...             ...   
1653      9                         0                     0              -1   
          10                        0                     0              -1   
          11                        0                     0               1   
          12                        0                     0              -1   
          13                        1                    -1               0   

                       five_p_cleav_compl_2  five_p_cleav_prob  \
instances time_points                                            
0         0                               0           0.999468   
          1                               0           0.998997   
          2                               0           0.998971   
          3                               0           0.999318   
          4                              -1           0.999742   
...                                     ...                ...   
1653      9                               1           0.992391   
          10                              0           0.000000   
          11                              0           0.000000   
          12                              1           0.762377   
          13                              0           0.705569   

                       five_p_cleav_ss  
instances time_points                   
0         0                          1  
          1                          1  
          2                          1  
          3                          1  
          4                          1  
...                                ...  
1653      9                          1  
          10                         0  
          11                         0  
          12                         1  
          13                         1  

[23156 rows x 6 columns]

In [7]:
X_convert=convert_collection(X, "np-list")

In [8]:
# Check type
get_type(X), get_type(X_convert)

('pd-multiindex', 'np-list')

In [9]:
y = np.concatenate((np.full((1, 827), 'cleav'), np.full((1, 827), 'non_cleav')), axis=None)
labelencoder = LabelEncoder()
y_transform = labelencoder.fit_transform(y)

In [10]:
y_transform

array([0, 0, 0, ..., 1, 1, 1])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_convert, y_transform, test_size=0.2, random_state=42)

In [12]:
from aeon.transformations.collection.convolution_based import Rocket, MiniRocket, MultiRocket, HydraTransformer
# rocket = Rocket(n_kernels=10000)
rocket = MultiRocket()
rocket.fit(X_train)
X_train_transform = rocket.transform(X_train)
print(X_train_transform.shape)

(1323, 49728)


In [13]:
from sklearn.linear_model import RidgeClassifierCV
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [14]:
X_test_transform = rocket.transform(X_test)
print(X_test_transform.shape)
print(classifier.score(X_test_transform, y_test))

(331, 49728)
0.797583081570997


In [15]:
# Using pipeline

In [16]:
rocket = RocketClassifier(n_kernels=1000)
rocket.fit(X_train, y_train)
y_pred = rocket.predict(X_test)
accuracy_score(y_test, y_pred)

0.8580060422960725

In [17]:
import datetime
print(f"This Notebook last end-to-end runs on: {datetime.datetime.now()}\n")

This Notebook last end-to-end runs on: 2025-07-15 14:06:50.778322

